In [69]:
import argparse
from PIL import Image, ImageDraw, ImageOps, ImageFont
from crisismmd_dataset import clean_text

from torch.utils.data.dataloader import DataLoader
from crisismmd_dataset import CrisisMMDataset, labels_task1, labels_task2

prediction_file = './prediction.csv'
task = 'task2'

class Args:
    pass
opt = Args()
opt.load_size = 228
opt.crop_size = 224
opt.max_dataset_size = 2147483648
opt.debug = True

test_set = CrisisMMDataset()
test_set.initialize(opt, phase='test', cat='all',
                    task=task)

predictions = open(prediction_file).readlines()
label_map = labels_task1 if task == 'task1' else labels_task2
inverse_map = {v: k for k, v in label_map.items()}

[*] 281 samples loaded.


In [107]:
import textwrap
def divide(text, length):
    # Devide text into multiple lines if too long
    result = []
    for line in textwrap.wrap(text, width=length):
        result.append(line)
    return "\n".join(result)

output_size = 224
textarea_size = 50

def show(idx, dataset):
    data = dataset[idx]
    with Image.open(data['path_image']) as imagePIL:
        image = imagePIL.resize((output_size, output_size))

        toShow = Image.new(image.mode, (output_size, output_size+textarea_size), (255, 255, 255))
        toShow.paste(image, (0, textarea_size))

        font = ImageFont.truetype('FONTS/arial.ttf', 9)
        text = clean_text(data['text'])

        caption = "Tweet: {} \nPrediction: {}".format(divide(text, 40), inverse_map[int(predictions[idx])])
        ImageDraw.Draw(toShow).text(
            (0, 0),     # position
            caption,
            (0, 0, 0),   # color
            font=font
        )
        toShow.show()

In [110]:
for i in range(10, 20):
    show(i, test_set)